In [1]:
import pandas as pd
import bs4 as bs
import requests
import time
from urllib.parse import urljoin

In [2]:
source = requests.get('http://www.debates.org/index.php?page=debate-transcripts').content

In [3]:
soup = bs.BeautifulSoup(source,'html.parser')

In [4]:
content = soup.find(id='content-sm')

In [5]:
debatelink = []

links = soup.find_all("a") # Find all elements with the tag <a>
for link in links:
    if "debate-transcript" in link.get("href"):
        debatelink.append("https://www.debates.org"+link.get("href"))

debatelink

['https://www.debates.org/voter-education/debate-transcripts/',
 'https://www.debates.org/voter-education/debate-transcripts/vice-presidential-debate-at-the-university-of-utah-in-salt-lake-city-utah/',
 'https://www.debates.org/voter-education/debate-transcripts/october-22-2020-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/september-29-2020-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/october-19-2016-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/october-9-2016-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/october-4-2016-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/september-26-2016-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/october-22-2012-the-third-obama-romney-presidential-debate/',
 'https://www.debates.org/voter-education/debate-transcripts/october-16-2012-the-seco

In [6]:
date_link = []

for link in debatelink:
    date_link.append(link.split("/")[5])

In [7]:
date_link

['',
 'vice-presidential-debate-at-the-university-of-utah-in-salt-lake-city-utah',
 'october-22-2020-debate-transcript',
 'september-29-2020-debate-transcript',
 'october-19-2016-debate-transcript',
 'october-9-2016-debate-transcript',
 'october-4-2016-debate-transcript',
 'september-26-2016-debate-transcript',
 'october-22-2012-the-third-obama-romney-presidential-debate',
 'october-16-2012-the-second-obama-romney-presidential-debate',
 'october-11-2012-the-biden-romney-vice-presidential-debate',
 'october-3-2012-debate-transcript',
 '2008-debate-transcript',
 '2008-debate-transcript-2',
 'october-7-2008-debate-transcrip',
 'october-15-2008-debate-transcript',
 'october-13-2004-debate-transcript',
 'october-8-2004-debate-transcript',
 'october-5-2004-transcript',
 'september-30-2004-debate-transcript',
 'october-17-2000-debate-transcript',
 'october-11-2000-debate-transcript',
 'october-5-2000-debate-transcript',
 'october-3-2000-transcript',
 'october-16-1996-debate-transcript',
 'oct

In [12]:
df = pd.DataFrame(columns=["date", "text", "link"])
debatelink = []

base_url = "https://www.debates.org"
page = requests.get(urljoin(base_url, "/voter-education/debate-transcripts/"))
soup = bs.BeautifulSoup(page.content, 'html.parser')

links = soup.find_all("a")
for link in links:
    if "debate-transcript" in link.get("href") and "Media" not in link.get("href"):
        debatelink.append(urljoin(base_url, link.get("href")))
        df.loc[len(df)] = [None, None, urljoin(base_url, link.get("href"))]
for link in df["link"]:
    try:
        page = requests.get(link)
        soup = bs.BeautifulSoup(page.content, 'html.parser')
        text = soup.find_all("p")
        text = [t.get_text() for t in text]
        text = " ".join(text)
        df.loc[df["link"] == link, "text"] = text

        date_link = link.split("/")[5:6] 
        date_link = "-".join(date_link[0].split("-")[:3])
        df.loc[df["link"] == link, "date"] = date_link
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {link}: {e}")

df

,date,text,link
0,,Unofficial transcripts of most presidential an...,https://www.debates.org/voter-education/debate...
1,vice-presidential-debate,Vice Presidential Debate at the University of ...,https://www.debates.org/voter-education/debate...
2,october-22-2020,Presidential Debate at Belmont University in N...,https://www.debates.org/voter-education/debate...
3,september-29-2020,Presidential Debate at Case Western Reserve Un...,https://www.debates.org/voter-education/debate...
4,october-19-2016,Presidential Debate at the University of Nevad...,https://www.debates.org/voter-education/debate...
...,...,...,...
92,october-22-1976,"\nOctober 22, 1976\n The Third Carter-Ford Pre...",https://www.debates.org/voter-education/debate...
93,september-26-1960,"\nSeptember 26, 1960\n The First Kennedy-Nixon...",https://www.debates.org/voter-education/debate...
94,october-7-1960,"\nOctober 7, 1960\n The Second Kennedy-Nixon P...",https://www.debates.org/voter-education/debate...
95,october-13-1960,"\nOctober 13, 1960\n The Third Kennedy-Nixon P...",https://www.debates.org/voter-education/debate...
